In [1]:
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pywt
from scipy.signal import stft
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [16]:
def valfunc(path, label):
    # Load the EEG data
    file_path = path
    raw = mne.io.read_raw_edf(file_path, preload=True)
    
    # Apply a bandpass filter to remove noise
    raw.filter(0.5, 50., fir_design='firwin')
    
    # Define epoch length and events
    epoch_length = 30  # in seconds
    sampling_rate = raw.info['sfreq']  # get the sampling rate
    epoch_samples = int(epoch_length * sampling_rate)
    
    # Create epochs
    events = mne.make_fixed_length_events(raw, duration=epoch_length)
    epochs = mne.Epochs(raw, events, tmin=0, tmax=epoch_length, baseline=None)
    
    # Extract data from epochs
    epoch_data = epochs.get_data()
    num_epochs = len(epoch_data)
    print(f"Number of epochs: {num_epochs}")
    
    # Decompose the EEG signal
    def decompose_signal(epoch, wavelet='db2', level=4):
        coeffs = pywt.wavedec(epoch, wavelet, level=level)
        return coeffs
    
    # Extract Time-Domain Statistical Features
    def extract_time_domain_features(coeffs):
        features = []
        for coeff in coeffs:
            features.extend([
                np.max(coeff),
                np.min(coeff),
                np.mean(coeff),
                np.std(coeff),
                np.var(coeff),
                np.sqrt(np.mean(coeff**2)),  # RMS
                np.mean((coeff - np.mean(coeff))**3) / np.std(coeff)**3,  # Skewness
                np.mean((coeff - np.mean(coeff))**4) / np.std(coeff)**4   # Kurtosis
            ])
        return features
    
    # Extract Frequency-Domain Features
    def extract_frequency_domain_features(epoch, fs, nperseg=256):
        f, t, Zxx = stft(epoch, fs, nperseg=nperseg)
        delta_power = np.mean(np.abs(Zxx[(f >= 0.5) & (f < 4)]))
        theta_power = np.mean(np.abs(Zxx[(f >= 4) & (f < 8)]))
        alpha_power = np.mean(np.abs(Zxx[(f >= 8) & (f < 13)]))
        beta_power = np.mean(np.abs(Zxx[(f >= 13) & (f < 30)]))
        gamma_power = np.mean(np.abs(Zxx[(f >= 30) & (f < 50)]))
        return [delta_power, theta_power, alpha_power, beta_power, gamma_power]
    
    # Combine Feature Extraction
    def extract_features(epoch, fs):
        coeffs = decompose_signal(epoch)
        time_features = extract_time_domain_features(coeffs)
        freq_features = extract_frequency_domain_features(epoch, fs)
        return np.concatenate((time_features, freq_features))
    
    # Extract features for all epochs
    all_features1 = np.array([extract_features(epoch[0], sampling_rate) for epoch in epoch_data])
    
    # Ensure the labels list matches the number of epochs
    labels1 = [label] * (num_epochs)  # Example pattern, adjust based on actual labeling
    if len(labels1) < num_epochs:
        labels1.extend([1] * (num_epochs - len(labels1)))  # Padding with 0s for simplicity
    return all_features1,labels1

In [27]:
#Using regular expression to settle path name
import re
def reg(path):
    return re.sub("insomnia 7th sem/","",path)

ins1label=1

In [28]:
#Storing values in a tuple

In [29]:
ins2vals=valfunc(reg("insomnia 7th sem/ins2 (1).edf"),ins1label)

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins2 (1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 25727999  =      0.000 ... 50249.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    6.5s finished


Not setting metadata
1675 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1675 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 1674


<h2>Ins2 to features and Labels</h2>

In [30]:
#Making dataframes to finally join to make a dataset
ins2labelsdf=pd.DataFrame(ins2vals[1])
ins2featuresdf=pd.DataFrame(ins2vals[0])

<h2>Ins3 to features and Labels</h2>

In [31]:
ins3label=1
ins3vals=valfunc(reg("insomnia 7th sem/ins3.edf"),ins3label)

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7910655  =      0.000 ... 30900.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


Not setting metadata
1030 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1030 events and 7681 original time points ...


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    2.0s finished


0 bad epochs dropped
Number of epochs: 1030


In [32]:
#Making dataframes to finally join to make a dataset
ins3labelsdf=pd.DataFrame(ins3vals[1])
ins3featuresdf=pd.DataFrame(ins3vals[0])

<h2>Ins4 to features and Labels</h2>

In [37]:
ins4label=1
ins4vals=valfunc(reg("insomnia 7th sem/ins4.edf"),ins4label)

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19553791  =      0.000 ... 38190.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.3s remaining:    0.0s


Not setting metadata
1273 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1273 events and 15361 original time points ...


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    2.7s finished


0 bad epochs dropped
Number of epochs: 1273


In [38]:
#Making dataframes to finally join to make a dataset
ins4labelsdf=pd.DataFrame(ins4vals[1])
ins4featuresdf=pd.DataFrame(ins4vals[0])

<h2>Ins5 to features and Labels</h2>

In [39]:
ins5label=1
ins5vals=valfunc(reg("insomnia 7th sem/ins5.edf"),ins5label)
ins5labelsdf=pd.DataFrame(ins5vals[1])
ins5featuresdf=pd.DataFrame(ins5vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins5.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 26419199  =      0.000 ... 51599.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    5.6s finished


Not setting metadata
1720 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1720 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 1719


<h2>Ins6 to features and Labels</h2>

In [40]:
ins6label=1
ins6vals=valfunc(reg("insomnia 7th sem/ins6.edf"),ins6label)
ins6labelsdf=pd.DataFrame(ins6vals[1])
ins6featuresdf=pd.DataFrame(ins6vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins6.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 24729599  =      0.000 ... 48299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    6.7s finished


Not setting metadata
1610 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1610 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 1609


<h2>Ins7 to features and Labels</h2>

In [41]:
ins7label=1
ins7vals=valfunc(reg("insomnia 7th sem/ins7.edf"),ins7label)
ins7labelsdf=pd.DataFrame(ins7vals[1])
ins7featuresdf=pd.DataFrame(ins7vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins7.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 22778879  =      0.000 ... 44489.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    5.2s finished


Not setting metadata
1483 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1483 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 1482


<h2>Ins8 to features and Labels</h2>

In [42]:
ins8label=1
ins8vals=valfunc(reg("insomnia 7th sem/ins8.edf"),ins8label)
ins8labelsdf=pd.DataFrame(ins8vals[1])
ins8featuresdf=pd.DataFrame(ins8vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins8.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15359999  =      0.000 ... 29999.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    6.1s finished


Not setting metadata
1000 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1000 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 999


<h2>Ins9 to features and Labels</h2>

In [43]:
ins9label=1
ins9vals=valfunc(reg("insomnia 7th sem/ins9.edf"),ins9label)
ins9labelsdf=pd.DataFrame(ins9vals[1])
ins9featuresdf=pd.DataFrame(ins9vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\ins9.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15780351  =      0.000 ... 30820.998 secs...


C:\Users\karti\AppData\Local\Temp\ipykernel_16240\3717673409.py:4: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_edf(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    6.3s finished


Not setting metadata
1027 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1027 events and 15361 original time points ...
0 bad epochs dropped
Number of epochs: 1027


<h2>n1 to features and Labels</h2>

In [44]:
n1label=0
n1vals=valfunc(reg("insomnia 7th sem/n1.edf"),n1label)
n1labelsdf=pd.DataFrame(n1vals[1])
n1featuresdf=pd.DataFrame(n1vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 17725439  =      0.000 ... 34619.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    6.5s finished


Not setting metadata
1154 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1154 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 1153


<h2>n2 to features and Labels</h2>

In [45]:
n2label=0
n2vals=valfunc(reg("insomnia 7th sem/n2.edf"),n2label)
n2labelsdf=pd.DataFrame(n2vals[1])
n2featuresdf=pd.DataFrame(n2vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 22579199  =      0.000 ... 44099.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    6.1s finished


Not setting metadata
1470 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1470 events and 15361 original time points ...
1 bad epochs dropped
Number of epochs: 1469


<h2>n3 to features and Labels</h2>

In [46]:
n3label=0
n3vals=valfunc(reg("insomnia 7th sem/n3.edf"),n3label)
n3labelsdf=pd.DataFrame(n3vals[1])
n3featuresdf=pd.DataFrame(n3vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15287295  =      0.000 ... 29857.998 secs...


C:\Users\karti\AppData\Local\Temp\ipykernel_16240\3717673409.py:4: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_edf(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    6.2s finished


Not setting metadata
995 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 995 events and 15361 original time points ...
0 bad epochs dropped
Number of epochs: 995


<h2>n4 to features and Labels</h2>

In [47]:
n4label=0
n4vals=valfunc(reg("insomnia 7th sem/n4.edf"),n4label)
n4labelsdf=pd.DataFrame(n4vals[1])
n4featuresdf=pd.DataFrame(n4vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n4.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7147999  =      0.000 ... 35739.995 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1321 samples (6.605 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s


Not setting metadata
1191 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1191 events and 6001 original time points ...


[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    2.2s finished


0 bad epochs dropped
Number of epochs: 1191


<h2>n5 to features and Labels</h2>

In [48]:
n5label=0
n5vals=valfunc(reg("insomnia 7th sem/n5.edf"),n5label)
n5labelsdf=pd.DataFrame(n5vals[1])
n5featuresdf=pd.DataFrame(n5vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n5.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 15222783  =      0.000 ... 29731.998 secs...


C:\Users\karti\AppData\Local\Temp\ipykernel_16240\3717673409.py:4: RuntimeWarning: Number of records from the header does not match the file size (perhaps the recording was not stopped before exiting). Inferring from the file size.
  raw = mne.io.read_raw_edf(file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    6.2s finished


Not setting metadata
991 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 991 events and 15361 original time points ...
0 bad epochs dropped
Number of epochs: 991


<h2>n6 to features and Labels</h2>

In [50]:
n6label=0
n6vals=valfunc(reg("insomnia 7th sem/n6.edf"),n6label)
n6labelsdf=pd.DataFrame(n6vals[1])
n6featuresdf=pd.DataFrame(n6vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n6.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4047999  =      0.000 ... 31624.992 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (6.602 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Not setting metadata
1054 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1054 events and 3841 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.5s finished


Number of epochs: 1054


<h2>n7 to features and Labels</h2>

In [51]:
n7label=0
n7vals=valfunc(reg("insomnia 7th sem/n7.edf"),n7label)
n7labelsdf=pd.DataFrame(n7vals[1])
n7featuresdf=pd.DataFrame(n7vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n7.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 3781119  =      0.000 ... 29539.992 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (6.602 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Not setting metadata
984 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 984 events and 3841 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.5s finished


Number of epochs: 984


<h2>n8 to features and Labels</h2>

In [52]:
n8label=0
n8vals=valfunc(reg("insomnia 7th sem/n8.edf"),n8label)
n8labelsdf=pd.DataFrame(n8vals[1])
n8featuresdf=pd.DataFrame(n8vals[0])

Extracting EDF parameters from C:\Users\karti\insomnia 7th sem\n8.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6007999  =      0.000 ... 30039.995 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1321 samples (6.605 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s


Not setting metadata
1001 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1001 events and 6001 original time points ...


[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    1.4s finished


0 bad epochs dropped
Number of epochs: 1001


<h1>Making Dataset</h1>

In [56]:
all_features=pd.concat([ins2featuresdf,ins3featuresdf,ins4featuresdf,ins5featuresdf,ins6featuresdf,ins7featuresdf,ins8featuresdf,ins9featuresdf,n1featuresdf,n2featuresdf,n3featuresdf,n4featuresdf,n5featuresdf,n6featuresdf,n7featuresdf,n8featuresdf],ignore_index=True)

In [57]:
all_features.shape

(19651, 45)

In [62]:
labelsall=pd.concat([ins2labelsdf,ins3labelsdf,ins4labelsdf,ins5labelsdf,ins6labelsdf,ins7labelsdf,ins8labelsdf,ins9labelsdf,n1labelsdf,n2labelsdf,n3labelsdf,n4labelsdf,n5labelsdf,n6labelsdf,n7labelsdf,n8labelsdf],ignore_index=True)

In [64]:
Dataset=pd.concat([all_features,labelsall],axis=1)

In [63]:
labelsall.shape

(19651, 1)

In [65]:
Dataset.shape

(19651, 46)

<h1>Dataset To Csv</h1>

In [67]:
Dataset.to_csv('InsomniafinalDataset.csv', index=False)